# **🎥 Procesamiento de Videos + 🖼️ Aumento de Datos**

Este notebook automatiza:

1.   📂 Extracción de frames (escala de grises + redimensión).

2.   ✨ Aumento de datos (transformaciones aleatorias para entrenar modelos de ML).





In [ ]:
pip install opencv-python albumentations numpy

## **🔧 Funciones Clave**

`extract_frames(video_path, output_folder, resolution=(640, 640))`

🚀 Qué hace:


*   Extrae todos los frames de un video.

*   Convierte a escala de grises (⚫⚪).

*   Redimensiona a 640x640 píxeles (configurable).

*   Guarda como PNG en una carpeta organizada.

📁 Estructura de salida:



```
frames_extracted/  
└── Sample1/  
    ├── Sample1_frame_0000.png  
    ├── Sample1_frame_0001.png  
    └── ...  
```

---

`augment_images(input_folder, output_folder, video_name, num_aug=5)`



*   🎲 Transformaciones aplicadas (aleatorias)

*   ↔️ Volteo horizontal (50% de probabilidad).

*   🔄 Rotación (±15°).

*   ☀️ Ajuste de brillo/contraste.

*   📸 Desenfoque (Motion Blur y Gaussian Blur).

---


**✂️ Salida:**


Genera 5 imágenes aumentadas por frame (ej: Sample1_frame_0000_aug_0.png).






In [ ]:
import cv2
import os
import numpy as np
import albumentations as A

# Función para extraer frames de un video
def extract_frames(video_path, output_folder, resolution=(640, 640)):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Convertir a escala de grises
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Redimensionar
        frame_resized = cv2.resize(frame_gray, resolution)
        # Guardar imagen con nombre único
        frame_filename = os.path.join(output_folder, f"{video_name}_frame_{frame_count:04d}.png")
        cv2.imwrite(frame_filename, frame_resized)
        frame_count += 1

    cap.release()
    print(f"Extraídos {frame_count} frames de {video_path}")

# Función para aplicar data augmentation a las imágenes extraídas
def augment_images(input_folder, output_folder, video_name, num_aug=5):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Definir el pipeline de augmentación
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.MotionBlur(blur_limit=3, p=0.3),
        A.GaussianBlur(blur_limit=3, p=0.3),
    ])

    for img_file in os.listdir(input_folder):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, img_file)
            # Leer la imagen en escala de grises
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            base_name = os.path.splitext(img_file)[0]

            for i in range(num_aug):
                augmented = transform(image=image)
                aug_image = augmented["image"]
                aug_filename = os.path.join(output_folder, f"{base_name}_aug_{i}_{video_name}.png")
                cv2.imwrite(aug_filename, aug_image)
    print(f"Imágenes aumentadas guardadas en {output_folder}")


## **⚡ Procesamiento de Videos**


In [ ]:
# Rutas de los videos
video_files = ["Sample1.mp4", "Sample5.mp4"]

# Carpetas para guardar los frames y los frames aumentados
frames_folder = "frames_extracted"
aug_folder = "frames_augmented"

# Extraer frames de cada video en una carpeta separada
for video in video_files:
    video_name = os.path.splitext(os.path.basename(video))[0]
    output_folder = os.path.join(frames_folder, video_name)
    extract_frames(video, output_folder, resolution=(640, 640))

# Aplicar data augmentation a los frames extraídos de cada video
for video in video_files:
    video_name = os.path.splitext(os.path.basename(video))[0]
    input_folder = os.path.join(frames_folder, video_name)
    output_aug_folder = os.path.join(aug_folder, video_name)
    augment_images(input_folder, output_aug_folder, video_name, num_aug=5)


Extraídos 84 frames de Sample1.mp4
Extraídos 107 frames de Sample5.mp4
Imágenes aumentadas guardadas en frames_augmented/Sample1
Imágenes aumentadas guardadas en frames_augmented/Sample5


## **📦 Compresión de Resultados**

In [ ]:
!zip -r frames_extracted.zip frames_extracted
!zip -r frames_augmented.zip frames_augmented